In [1]:
import speech_recognition as sr
from transformers import AutoTokenizer, AutoModelForCausalLM
import pyttsx3
import torch

/Users/erictommathews/miniforge3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [17]:
# Initialize speech recognition and text-to-speech
recognizer = sr.Recognizer()
recognizer.dynamic_energy_threshold = False  # Ensure the timeout is respected
engine = pyttsx3.init()

# Change the voice and adjust the speech rate
voices = engine.getProperty('voices')
voice_id = 'com.apple.speech.synthesis.voice.samantha.premium'
engine.setProperty('voice', voice_id)
engine.setProperty('rate', 190)

In [18]:
speak("Hello, I am a chatbot. How can I help you today?")

In [19]:
import ollama
# response = ollama.chat(model='llama2', messages=[
#   {
#     'role': 'user',
#     'content': 'Why is the sky blue?',
#     'context': ["always answer in 2 sentences"]
#   },
# ])
# print(response['message']['content'])

In [20]:
def listen(timeout=5, phrase_time_limit=5):
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=timeout, phrase_time_limit=phrase_time_limit)
            text = recognizer.recognize_google(audio)
            return text.lower()
        except sr.WaitTimeoutError:
            print("Listening timed out after 10 seconds")
            return ""
        except sr.UnknownValueError:
            print("Could not understand audio")
            return ""
        except sr.RequestError as e:
            print(f"Could not request results; {e}")
            return ""

In [21]:
def speak(text):
    engine.say(text)
    engine.runAndWait()

In [22]:
def generate_response(text):
    response = ollama.chat(model='llama2', messages=[
        {
            'role': 'user',
            'content': text+". Answer always in no more than 2 sentences",
        },
    ])
    return response['message']['content']

In [23]:
import time
def main():
    assistant_name = "jarvis"
    speak(f"Hello, I'm {assistant_name}. How can I help you?")
    
    while True:
        print("Listening... (Say 'exit' to stop)")
        user_input = listen(timeout=5)
        print(f"You said: {user_input}")
        
        if user_input:  # Check if user_input is not empty
            if "exit" in user_input.lower():
                speak("Goodbye!")
                break
            elif assistant_name in user_input.lower():
                user_input = user_input.replace(assistant_name, "").strip()
                response = generate_response(user_input)
                print(response)
                #wait 2 seconds before speaking
                time.sleep(2)
                speak(response)
            else:
                print("I'm waiting for my name. Say 'Jarvis' to get my attention.")
        else:
            print("I didn't catch that. Please try again.")

In [24]:
main()

Listening... (Say 'exit' to stop)
Listening...
You said: hi jarvis how are you today

I'm good, thanks for asking! I'm feeling energized and ready to take on the day.
Listening... (Say 'exit' to stop)
Listening...
You said: thanks jarvis exit
